**1.0 Data Preparation (non-rerunnable, privacy-restriced)**

The comments have been extracted and turned into a dataset, accessible via google drive. This part reviews the data and prepares it by removing any deleted comments, adding a stable index and anonymising usernames for privacy reasons. The stable index ensures consistency during the thematic coding, especially considering the dataset may be reloaded which causes default indexes to change.

For privacy reasons, this part of the data analysis is not rerunable in Github, as the file contains the original authornames. For rerunning the code, please start from step **1.1 GitHub-Ready Code**.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/Dataprojects/Methodology/dataset_tiktok-comments-scraper_2025-11-04_02-52-04-694.csv"
df_new = pd.read_csv(file_path)

print(f"✅ Loaded {len(df_new):,} rows from {file_path}")
pd.set_option('display.max_colwidth', None)
df_new['text'].head()

In [ ]:
# Drop NaN in 'text'
rows_before_drop = len(df_new)
df_new = df_new.dropna(subset=['text']).copy()
rows_after_drop = len(df_new)

print(f"✅ Initial rows: {rows_before_drop:,}")
print(f"🗑️ Rows removed due to NaN in 'text': {rows_before_drop - rows_after_drop:,}")
print(f"✨ Rows remaining: {rows_after_drop:,}")

if "text" in df_new.columns and "comment" not in df_new.columns:
    df_new.rename(columns={"text": "comment"}, inplace=True)
    print("📝 Renamed 'text' to 'comment'.")

if "diggCount" in df_new.columns and "likes" not in df_new.columns:
    df_new.rename(columns={"diggCount": "likes"}, inplace=True)
    print("📝 Renamed 'diggCount' to 'likes'.")

df_new["comment"] = df_new["comment"].astype(str)
df_new = df_new[df_new["comment"].str.strip().str.len() > 0].copy()

# Create anonymized user column
if "uid" in df_new.columns and "anon_user" not in df_new.columns:
    unique_uids = df_new["uid"].dropna().unique()
    uid_to_anon = {uid: f"User_{i+1}" for i, uid in enumerate(unique_uids)}
    df_new["anon_user"] = df_new["uid"].map(uid_to_anon)
    print(f"👤 Created 'anon_user' for {len(unique_uids):,} users.")

# ✅ Privacy: drop identifying columns AFTER anon_user is created
cols_to_drop = [c for c in ["author", "uid"] if c in df_new.columns]
if cols_to_drop:
    df_new.drop(columns=cols_to_drop, inplace=True)
    print(f"🔒 Dropped identifying columns for privacy: {cols_to_drop}")

# add stable index
df_new = df_new.reset_index(drop=True)

if "stable_index" not in df_new.columns:
    df_new["stable_index"] = df_new.index
    print("📌 Added stable 'stable_index' column.")

print("\nFinal TikTok cleaned dataframe info:")
df_new.info()

clean_tiktok_path = "/content/drive/MyDrive/Dataprojects/Methodology/tiktok_cleaned_comments.csv"
df_new.to_csv(clean_tiktok_path, index=False, encoding="utf-8")
print(f"\n💾 Cleaned TikTok dataset saved to:\n{clean_tiktok_path}")


**1.1 GitHub-Ready Code**

In [ ]:
# --- Fresh clone (no cache) ---
import shutil
from pathlib import Path
import os

REPO = "data-analysis-TikTok"
REPO_URL = "https://github.com/salomevanzutphen/data-analysis-TikTok.git"

if Path(REPO).exists():
    shutil.rmtree(REPO)

!git clone {REPO_URL}
os.chdir(REPO)

!ls -lh data/dataset/tiktok_cleaned_comments.csv


In [ ]:
import pandas as pd

df_tiktok = pd.read_csv("data/dataset/tiktok_cleaned_comments.csv")
print(df_tiktok.shape)
df_tiktok.head()


In [ ]:
from pathlib import Path
import pandas as pd

CANDIDATE_PATHS = [
    Path("data/dataset/tiktok_cleaned_comments.csv"),
    Path("../data/dataset/tiktok_cleaned_comments.csv"),
]

CLEAN_TIKTOK_DATA = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if CLEAN_TIKTOK_DATA is None:
    raise FileNotFoundError(
        "Could not find tiktok_cleaned_comments.csv.\n"
        "Expected it at data/dataset/tiktok_cleaned_comments.csv"
    )

df_tiktok = pd.read_csv(CLEAN_TIKTOK_DATA)
print(f"✅ Loaded {len(df_tiktok):,} TikTok rows from: {CLEAN_TIKTOK_DATA}")


**1.2 Reviewing Duplicate Comments**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from IPython.display import display

df_tiktok = pd.read_csv(CLEAN_TIKTOK_DATA)
print(f"Loaded {len(df_tiktok):,} TikTok rows")

text_col = "comment"

df_tiktok["createTimeISO"] = pd.to_datetime(df_tiktok["createTimeISO"], errors="coerce")

duplicate_counts = df_tiktok[text_col].value_counts()
duplicate_texts = duplicate_counts[duplicate_counts > 1].index.tolist()

print(f"🔍 Found {len(duplicate_texts)} duplicated unique TikTok comments.")

duplicates_full = df_tiktok[df_tiktok[text_col].isin(duplicate_texts)].copy()

def summarize_group(g):
    g = g.sort_values("createTimeISO")
    freq = len(g)

    rows = []
    for _, r in g.iterrows():
        rows.append(
            f"user={r.get('anon_user', '')}, "
            f"likes={r.get('likes', '')}, "
            f"createdAt={r.get('createTimeISO', '')}"
        )

    return pd.Series({
        "frequency": freq,
        "instances": "\n".join(rows)
    })


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    summary_tiktok = duplicates_full.groupby(text_col).apply(summarize_group).reset_index()


summary_tiktok = summary_tiktok.sort_values("frequency", ascending=False).reset_index(drop=True)


summary_tiktok.index = summary_tiktok.index + 1

# Display
pd.set_option("display.max_colwidth", None)

display(
    summary_tiktok.style.set_properties(
        subset=[text_col],
        **{
            "white-space": "normal",
            "max-width": "300px",
        }
    ).set_properties(
        subset=["instances"],
        **{
            "white-space": "pre-wrap",
            "max-width": "1000px",
            "font-family": "monospace"
        }
    )
)


**2.0 Opinion leaders**

The opinion leaders are identified using the benchmark of 75% of the total likecount/upvotes of the overall dataset. After knowing how many comments are in this subset, they are reviewed with their ranking, stable index, like count and text.

In [ ]:
import pandas as pd

df = pd.read_csv(CLEAN_TIKTOK_DATA)

LIKES_COL = "likes"

if LIKES_COL not in df.columns:
    raise ValueError(f"Column '{LIKES_COL}' not found. Available columns: {list(df.columns)}")

df = df[df[LIKES_COL].notna() & (df[LIKES_COL] >= 0)]


total_comments = len(df)
total_likes = df[LIKES_COL].sum()


if total_comments == 0 or total_likes == 0:
    print("No comments or no likes in the dataset.")
else:
    target_likes_75 = 0.75 * total_likes

    df_sorted = df.sort_values(LIKES_COL, ascending=False).reset_index(drop=True)
    df_sorted["cum_likes"] = df_sorted[LIKES_COL].cumsum()

    first_row_reaching_75 = df_sorted[df_sorted["cum_likes"] >= target_likes_75].index.min()
    comments_needed = int(first_row_reaching_75) + 1

    percent_of_dataset = (comments_needed / total_comments) * 100

    print(f"Total comments: {total_comments:,}")
    print(f"Total likes: {total_likes:,}")
    print(f"75% benchmark of total likes: {target_likes_75:,.0f}")
    print(f"Comments needed to reach 75% of likes: {comments_needed:,}")
    print(f"These comments represent {percent_of_dataset:.2f}% of the dataset.")


In [ ]:
import pandas as pd

df = pd.read_csv(CLEAN_TIKTOK_DATA)

opinionleaders_tiktok = (
    df.sort_values('likes', ascending=False)
      .head(35)
      .copy()
)

opinionleaders_tiktok["stable_index"] = opinionleaders_tiktok.index

opinionleaders_tiktok.index = range(1, len(opinionleaders_tiktok) + 1)


opinionleaders_tiktok = opinionleaders_tiktok[["likes", "stable_index", "comment"]]

styled_table = opinionleaders_tiktok.style.set_properties(
    subset=["comment"],
    **{
        "white-space": "normal",
        "word-wrap": "break-word",
        "width": "550px"
    }
)

styled_table


**2.1 Thematic Analysis**

This part thematically codes each comment within the subset of opinion leaders. The following categorizations have been identified:

-  **Humor**: Jokes that involve self-comparison (in reference to the models)
-   **Overall Review**: Overall appreciation, with minimal context
-   **Ideal Modelling Standard**: Discussion about the expectations and ideals of a model
-   **Model Praise**: Naming or referring to specific models and appreciation for their performance/beauty/presence.


***Artists***
-   **Missy Elliot**: mixed sentiment
-   **Twice**: Mostly negative feedback
-   **Karol G**: Overwhelmingly positive
-   **Madison Beer**: Generally praised





Below follows the thematic distribution of comments and likecounts, visualised in a pie chart. Furthermore, all comments and their related themes are be provided as well.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# --- Load data ---
df_clean = pd.read_csv(CLEAN_TIKTOK_DATA)
print(f"Loaded TikTok dataset with {len(df_clean):,} rows.")

if "stable_index" not in df_clean.columns:
    df_clean["stable_index"] = df_clean.index
    print(
        "⚠️ 'stable_index' not found – created from current index. "
        "Make sure this matches how you chose indices for themes_by_index."
    )

themes_by_index = {
    "Humor": [633, 637],
    "Model Praise": [546, 274, 62, 55, 56, 53, 288],
    "Ideal Modelling Standard": [57, 58, 179, 63, 64, 65, 59, 56, 53],
    "Missy Elliot": [1661],
    "Twice": [762, 763, 760, 774, 783, 766, 169, 779, 769],
    "Karol G": [538, 767, 777, 542, 182, 761, 765],
    "Madison Beer": [765, 771],
    "Overall Review": [171],
}

text_col_name = None
for col in ["text", "content", "body", "comment"]:
    if col in df_clean.columns:
        text_col_name = col
        break
if text_col_name is None:
    raise ValueError("No text column found (expected 'text', 'content', 'body', or 'comment').")

likes_col_name = "likes"
if likes_col_name not in df_clean.columns:
    raise ValueError(f"Likes column '{likes_col_name}' not found in df_clean.")

subset = df_clean[["stable_index", text_col_name, likes_col_name]].copy()
subset = subset.rename(columns={text_col_name: "comment", likes_col_name: "likeCount"})

subset["likeCount"] = subset["likeCount"].fillna(0).astype(int)
subset["comment"] = subset["comment"].astype(str).fillna("").str.strip()

top_comments_df = (
    subset.sort_values("likeCount", ascending=False)
          .head(35)[["stable_index", "comment", "likeCount"]]
          .copy()
)

ranked = top_comments_df.reset_index(drop=True).copy()

index_to_themes = {}
for theme, indices in themes_by_index.items():
    for idx in indices:
        index_to_themes.setdefault(idx, []).append(theme)

def join_labels(idx):
    labs = index_to_themes.get(idx, [])
    return ", ".join(labs) if labs else ""

ranked["themes"] = ranked["stable_index"].apply(join_labels)

missing = ranked.loc[ranked["themes"].eq(""), "stable_index"].tolist()
if missing:
    print(f"⚠️ stable_index values in top-35 with NO theme label: {missing}")
else:
    print("✅ All top-35 TikTok comments have at least one theme label.")

ranked["theme_list"] = ranked["themes"].apply(
    lambda x: x.split(", ") if isinstance(x, str) and x else []
)

long = ranked.explode("theme_list").rename(columns={"theme_list": "theme"})
long = long[long["theme"] != ""]

counts = long.groupby("theme")["stable_index"].nunique().sort_values(ascending=False)
likes = long.groupby("theme")["likeCount"].sum().sort_values(ascending=False)

total_likes = likes.sum()

summary_df = pd.DataFrame({
    "Comments": counts,
    "Likes": likes,
    "Share of Likes (%)": (likes / total_likes * 100).round(1),
}).sort_values("Likes", ascending=False)

print("\n📊 THEME SUMMARY (Top 35 TikTok Comments)\n")
print(summary_df.to_string())

plt.figure(figsize=(10, 7))

wedges, texts, autotexts = plt.pie(
    summary_df["Likes"],
    labels=None,
    autopct="%1.1f%%",
    startangle=140,
    pctdistance=0.75
)

plt.title("Thematic Engagement Distribution")

plt.legend(
    wedges,
    summary_df.index,
    title="Theme",
    loc="center left",
    bbox_to_anchor=(1.02, 0.5),
    frameon=False
)

plt.tight_layout()
plt.show()

# --- GitHub-ready output path ---
OUTPUT_DIR = Path("data/processed")
if not OUTPUT_DIR.exists():
    alt = Path("../data/processed")
    if alt.parent.exists():
        OUTPUT_DIR = alt

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_PATH = OUTPUT_DIR / "top35_tiktok_comments_labeled_by_theme.csv"
ranked.to_csv(OUT_PATH, index=False, encoding="utf-8")
print(f"\n📁 Saved labeled top-35 TikTok comments to: {OUT_PATH}")


In [ ]:
import pandas as pd
import textwrap
from pathlib import Path


CANDIDATE_PATHS = [
    Path("data/processed/top35_tiktok_comments_labeled_by_theme.csv"),
    Path("../data/processed/top35_tiktok_comments_labeled_by_theme.csv"),
]

LABELED_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if LABELED_PATH is None:
    raise FileNotFoundError(
        "Could not find top35_tiktok_comments_labeled_by_theme.csv.\n"
        "Run the previous code block first to generate it.\n"
        "Expected: data/processed/top35_tiktok_comments_labeled_by_theme.csv"
    )

df_labeled_comments = pd.read_csv(LABELED_PATH)
print(f"✅ Loaded labeled TikTok comments from: {LABELED_PATH}")

if "index" in df_labeled_comments.columns:
    df_labeled_comments = df_labeled_comments.rename(columns={"index": "stable_index"})

df_labeled_comments["themes"] = df_labeled_comments["themes"].apply(
    lambda x: [theme.strip() for theme in x.split(",")] if pd.notna(x) and x else []
)

all_unique_themes = sorted(
    {theme for sublist in df_labeled_comments["themes"] for theme in sublist}
)

MAX_WIDTH = 100

print("\n=== TikTok Comments by Theme (Top 35) ===\n")

for theme in all_unique_themes:
    print(f"\n--- 🔵 Theme: {theme} ---")

    theme_comments = df_labeled_comments[
        df_labeled_comments["themes"].apply(lambda x: theme in x)
    ]

    if theme_comments.empty:
        print(f"  No comments found for '{theme}'.")
        continue

    theme_comments = theme_comments.sort_values(by="likeCount", ascending=False)

    for i, row in enumerate(theme_comments.itertuples(index=False), start=1):
        wrapped_comment = textwrap.fill(
            row.comment,
            width=MAX_WIDTH,
            break_long_words=False,
            replace_whitespace=False,
        )
        print(f"  {i}. 👍 {row.likeCount} likes  |  Index: {row.stable_index}")
        print(f"     {wrapped_comment}")
        print("-" * MAX_WIDTH)


**3.0 Sentiment Analysis**

This includes a sentiment analysis on the overall dataset, as well as a manual review of 100 comments. Here, the doubtful or incorrect classifications are separated for closer review, before calculating the accuracy rate.  

In [ ]:
!pip install -q transformers torch

import pandas as pd
import torch
import time
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# INPUT
df = pd.read_csv(CLEAN_TIKTOK_DATA)

# OUTPUT (GitHub/Colab friendly)
OUTPUT_DIR = Path("data/processed")
if not OUTPUT_DIR.exists():
    alt = Path("../data/processed")
    if alt.parent.exists():
        OUTPUT_DIR = alt

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
SENTIMENT_OUTPUT_PATH = OUTPUT_DIR / "cleaned_tiktok_data_sentiment.csv"


text_col = next((c for c in ["comment", "text", "content", "body"] if c in df.columns), None)
if text_col is None:
    raise ValueError("No text column found (expected one of: comment, text, content, body).")

df = df[~df[text_col].isin(["[deleted]", "[removed]"])].reset_index(drop=True)
df[text_col] = df[text_col].fillna("").astype(str)

print(f"✅ Loaded {len(df):,} TikTok comments")
print(f"📝 Using text column: {text_col}")

# Model
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    model_max_length=512,
    truncation=True,
    padding_side="right"
)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = 0 if torch.cuda.is_available() else -1
sentiment_pipeline = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=device
)


texts = df[text_col].tolist()
batch_size = 128
results = []

print("\n🔄 Running 3-class sentiment (pos/neu/neg) with truncation to 512 tokens...\n")
for i in range(0, len(texts), batch_size):
    batch = texts[i:i + batch_size]
    out = sentiment_pipeline(batch, truncation=True, padding=True, max_length=512)
    results.extend(out)

    if (i // batch_size) % 10 == 0:
        print(f"Processed {min(i + batch_size, len(texts))}/{len(texts)}", end="\r")

    time.sleep(0.01)

print("\n✅ Sentiment analysis complete!")

df["sentiment_label"] = [r["label"].upper() for r in results]
df["sentiment_score"] = [r["score"] for r in results]


df.to_csv(SENTIMENT_OUTPUT_PATH, index=False, encoding="utf-8")
print(f"💾 Saved TikTok sentiment file to:\n{SENTIMENT_OUTPUT_PATH}")

df[[text_col, "sentiment_label", "sentiment_score"]].head(5)


In [ ]:
import pandas as pd
from pathlib import Path

CANDIDATE_PATHS = [
    Path("data/processed/cleaned_tiktok_data_sentiment.csv"),
    Path("../data/processed/cleaned_tiktok_data_sentiment.csv"),
]

TIKTOK_SENTIMENT = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if TIKTOK_SENTIMENT is None:
    raise FileNotFoundError(
        "Could not find cleaned_tiktok_data_sentiment.csv.\n"
        "Run the sentiment generation block first.\n"
        "Expected: data/processed/cleaned_tiktok_data_sentiment.csv"
    )

df_tiktok = pd.read_csv(TIKTOK_SENTIMENT)
print(f"Loaded {len(df_tiktok):,} TikTok rows from: {TIKTOK_SENTIMENT}")



In [ ]:
import pandas as pd

df_loaded = pd.read_csv(TIKTOK_SENTIMENT)

text_col = next(
    (c for c in ["text", "content", "body", "comment"] if c in df_loaded.columns),
    None
)

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", None)

df_loaded[[text_col, "sentiment_label", "sentiment_score"]].head(100)


In [ ]:
import pandas as pd

df = pd.read_csv(TIKTOK_SENTIMENT)

text_col = next((c for c in ["comment","text","content","body"] if c in df.columns), None)
if text_col is None:
    raise ValueError("No text column found in dataset.")

indices = [1, 6, 8, 11, 15, 16, 21, 26, 32, 51, 54, 69, 70, 71, 75, 81, 86, 89, 90, 97]

df.loc[indices, [text_col, "sentiment_label", "sentiment_score"]]


**Review of misclassifications**


This dataset includes a lot of slang, like ‘ ate’, ‘eat down’ and ‘ ate hard’, which is a complimentary reference yet wrongly classified as negative. Comments where people expressed their aspirations for the show, like ‘only who can WALK’ being desired, not ‘an average guy playing next to LeBron’, subtly highlight the inadequacies of the show, therefore better suited as a negative sentiment, although the current neutral sentiment is not completely wrong either. Furthermore, many comments included mixed emotions, expressing a positive review yet wondering why there are so many negative responses. Technically this would be a positive sentiment, yet it is classified as negative. Then there are some comments with little to no context, naming the models with some emoji’s, or including expressions like ‘my jaw was on the floor’, or ‘i screamed’ which could be interpreted in different ways. This makes categorisation difficult, as more contextual awareness is needed. Therefore, classifications of positive or negative seem unreliable.



*Accuracy rate of 80%*

**3.1 Sentiment Popularity**

This reviews the popularity of each sentiment, first by reviewing their overall presence in the dataset, the top comments of each sentiment and the average like count per category.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(TIKTOK_SENTIMENT)
print(f"✅ Loaded {len(df):,} rows with sentiment data")


counts = df["sentiment_label"].value_counts()
percentages = (counts / counts.sum() * 100).round(2)

plt.figure(figsize=(6, 4))
bars = plt.bar(counts.index, counts.values, color=["#E74C3C", "#2ECC71", "#F1C40F"])

plt.title("Tik Tok Sentiment Distribution", fontsize=13, pad=10)
plt.xlabel("Sentiment Category", fontsize=11)
plt.ylabel("Number of Comments", fontsize=11)
plt.xticks(rotation=0)


for bar, pct in zip(bars, percentages):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + counts.max() * 0.01,
        f"{pct:.1f}%",
        ha="center",
        va="bottom",
        fontsize=10,

    )

plt.tight_layout()
plt.show()

print("\n📊 Sentiment Breakdown:")
for label, pct in percentages.items():
    print(f"{label}: {pct:.2f}% ({counts[label]} comments)")


In [ ]:
import pandas as pd


df_sentiment = pd.read_csv(TIKTOK_SENTIMENT)
df_sentiment.columns = df_sentiment.columns.str.lower()


text_col = next((c for c in ["comment","text","content","body"] if c in df_sentiment.columns), None)
if text_col is None:
    raise ValueError("No comment column found.")

df_sentiment = df_sentiment.rename(columns={
    text_col: "comment",
    "diggcount": "likes"
})


df_sentiment["likes"] = pd.to_numeric(df_sentiment["likes"], errors="coerce").fillna(0).astype(int)


df_sentiment["sentiment_label"] = df_sentiment["sentiment_label"].astype(str).str.lower()


if "sentiment_score" in df_sentiment.columns:
    df_sentiment = df_sentiment.rename(columns={"sentiment_score": "confidence"})


def print_top(df, sentiment_type, icon):
    subset = (
        df[df["sentiment_label"] == sentiment_type]
        .sort_values("likes", ascending=False)
        .head(15)
    )

    print(f"\n{icon} Top 15 Most-Liked {sentiment_type.capitalize()} Comments:\n")
    for _, row in subset.iterrows():
        snippet = str(row["comment"])[:500]
        print(f"💬 {snippet}")
        print(f"   🔹 Sentiment: {row['sentiment_label']} "
              f"({row['confidence']:.3f}) | Likes: {row['likes']}\n")

print_top(df_sentiment, "negative", "🔴")
print_top(df_sentiment, "positive", "🟢")
print_top(df_sentiment, "neutral", "🟡")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


df = pd.read_csv(TIKTOK_SENTIMENT)

df.columns = df.columns.str.lower()


text_col = next((c for c in ["comment","text","content","body"] if c in df.columns), None)
if text_col is None:
    raise ValueError("No comment column found.")


df = df.rename(columns={
    text_col: "comment",
    "diggcount": "likes"
})

df["likes"] = pd.to_numeric(df["likes"], errors="coerce").fillna(0)

df["sentiment_label"] = df["sentiment_label"].astype(str).str.lower()


df["sentiment_label_plot"] = df["sentiment_label"].str.title()

sentiment_stats = (
    df.groupby("sentiment_label_plot")["likes"]
      .agg(["mean", "sum", "count"])
      .reset_index()
      .sort_values("mean", ascending=False)
)

# Colors for chart
color_map = {
    "Positive": "#2ECC71",
    "Neutral": "#F1C40F",
    "Negative": "#E74C3C"

}
colors = [color_map.get(label, "gray") for label in sentiment_stats["sentiment_label_plot"]]

plt.figure(figsize=(7,4))
bars = plt.bar(sentiment_stats["sentiment_label_plot"], sentiment_stats["mean"], color=colors)
plt.title("Average Likes per Sentiment Category (Tik Tok)")
plt.ylabel("Average Likes per Comment")
plt.xlabel("Sentiment")

y_pad = sentiment_stats["mean"].max() * 0.01
for bar in bars:
    plt.text(
        bar.get_x() + bar.get_width()/2,
        bar.get_height() + y_pad,
        f"{bar.get_height():.0f}",
        ha="center",
        va="bottom",
        fontsize=10
    )

plt.tight_layout()
plt.show()

print("\n📊 Sentiment Popularity Summary:")
for _, row in sentiment_stats.iterrows():
    print(f"{row['sentiment_label_plot']}: {int(row['count'])} comments | "
          f"Avg Likes: {row['mean']:.1f} | Total Likes: {row['sum']:.0f}")


**4.0 Text Frequency Analysis**

This part includes an analysis of the most frequent words in the dataset. From the most frequent or thematically relevant words, the top 10 most liked comments are reviewed, to gain deeper contextual awareness.

In [ ]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import display

nltk.download("stopwords", quiet=True)

df = pd.read_csv(CLEAN_TIKTOK_DATA)
print(f"✅ Loaded {len(df):,} rows from {CLEAN_TIKTOK_DATA}")

text_col = next((c for c in ["comment", "text", "content", "body"] if c in df.columns), None)
if text_col is None:
    raise ValueError("❌ No text column found (expected one of: comment, text, content, body).")

df[text_col] = df[text_col].fillna("").astype(str).str.strip()
df = df[df[text_col].str.len() >= 5].copy()
print(f"🧹 Kept {len(df):,} comments with ≥5 characters")

stop_words = set(stopwords.words("english"))

project_stopwords = {"dont", "thats", "didnt", "like", "think", "would"}
stop_words |= project_stopwords

junk_tokens = {"s", "t", "m", "g", "1", "000", "la", "de", "se", "las", "el", "que"}
stop_words |= junk_tokens


vectorizer = CountVectorizer(
    stop_words=list(stop_words),
    lowercase=True,
    token_pattern=r"(?u)\b[a-zA-Z]{2,}\b"
)

X = vectorizer.fit_transform(df[text_col])
counts = X.sum(axis=0).A1
words = vectorizer.get_feature_names_out()

top_words = (
    pd.DataFrame({"word": words, "count": counts})
      .sort_values("count", ascending=False)
      .head(15)
      .reset_index(drop=True)
)

top_words.index = top_words.index + 1

print("\nTop 15 Words in TikTok Dataset:")
display(
    top_words.style.set_properties(**{
        "border": "1px solid black",
        "text-align": "left",
    })
)


In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 100)

TOP_N_WORDS = 15
TOP_N_COMMENTS = 15

for word in top_words["word"].head(TOP_N_WORDS):
    subset = df[
        df[text_col].str.contains(rf"\b{word}\b", case=False, regex=True)
    ]

    if subset.empty:
        continue

    top_comments = (
        subset.sort_values("likes", ascending=False)
              .head(TOP_N_COMMENTS)[[text_col, "likes", "stable_index"]]
              .reset_index(drop=True)
    )

    print(f"\n🔹 Top {TOP_N_COMMENTS} most-liked comments containing '{word}':\n")
    display(top_comments)
